# Tensorflow notes

For large datasets, instalation of cuDNN (NVIDIAs Deep Neural Network library) is needed.

Keras supports 3 different types of backends:
 - TensorFlow
 - Theano
 - CNTK
 
For saving keras models on disk: HDF5 and h5py

The samples for keras has to be a numpy array or a list of numpy arrays

The labels has to be a numpy array

Training the model: Optimizing the weights

Stochastic Gradient Descent (SGD):
 - the most widely known optimizer
 - Objective: minimize the loss function (like mean squared error)
 cat 0.75, dog 0.25: error = 0 - 0.25 = 0.25

- Overfitting: good at classifying the train data, but not good at classifying the test data
- How do we know: when validation << training
- Fighting again overfitting: more data and data augmentation (crop, zoom, rotating, flipping)
- and reducing the complexity of the model (reducing layers or neurons from layers)

### To scale data (from 0 to 1):

In [ ]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
data = [23, 40, 12, 1, 0]
n_data = np.array(data)
scaler = MinMaxScaler(feature_range=(0,1))

scaled_data = scaler.fit_transform((n_data).reshape(-1,1))
scaled_data

### Creating a model

1.

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, input_shape=(1,), activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

2.

In [ ]:
model = tf.keras.models.Sequential() # most common model
model.add(Dense(32, input_shape=(10,), activation='relu')) # The hidden layer: Dense is the most common one
model.add(Dense(2, activation='softmax')) # the output layer

### Check a model's details:

In [ ]:
model.summary()

### Compiling a model

In [ ]:
model.compile(tf.keras.optimizer.Adam(lr=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

or

In [ ]:

model.loss = 'sparse_categorical_crossentropy'
model.optimizer.lr = 0.0001

- the optimizer = Adam
- loss function: mse mean squared error
- lr = learning rate between 0.01 and 0.0001
- metrics: what's printed out when the model it's trained

#### Supervised learning
 - Labeled data

#### Unsupervised learning
 - clustering

#### Semi-supervised learning
 - When only part of the data is labeled
 - Pseudo-labeling: Train with the labeled data -> label using the model the unlabeled data -> train the whole model with the new 100% labeled data

### Fitting a model

In [ ]:
model.fit(scaled_data, train_labels, batch_size=10, shuffle=True, verbose=2)


- batch_size = how many piceces of data to be sent to the model at once
- how much output we want to see when we train the model

### To chose the validation set:

1. 

In [ ]:
model.fit(train_set, train_labels, validation_split=0.2, batch_size=10, epochs=20, shuffle=True, verbose=1)
# and the valid_set has to be this format:


2. We have explicitly the validation set:

In [ ]:
model.fit(train_set, train_labels, validation_data=valid_set, batch_size=10, epochs=20, shuffle=True, verbose=1)

And the validation data has to be this format:

In [ ]:
valid_set = [(sample, label), (sample, label) ... (sample, label)]

### Create a validation set
 - 1st way

In [ ]:
valid_set = [(sample, label), ... , (sample, label)]
model.fit(scaled_data, train_labels, validation_data=valid_set, batch_size=10, shuffle=True, verbose=2)
model.fit(scaled_data, train_labels, validation_split=0.1, batch_size=10, shuffle=True, verbose=2)

### Make a prediction
 - Classic prediction:

In [ ]:
predictions = model.predict(test_data, batch_size=10, verbose=0)

 - Rounded prediction:

In [ ]:
rounded_prediction = model.predict_classes(test_data, batch_size=10, verbose=0)

### Save a model classic

In [ ]:
model.save('my_model.h5')

It saves:
- the architecture of the model
- the weights
- the training configuration(compile): loss, optimizer
- the state of the optimizer (resume training)


### Save a model as json string:
- it saves only the architecture


In [ ]:
model.to_json()

### Load the model:

In [ ]:
new_model = tf.keras.models.load_model('my_model.h5')
# or
new_model = tf.keras.models.model_from_json(json_string)

### See the weights:

In [ ]:
model.get_weights()

### Prepare a CNN data(images)

In [ ]:
import tensorflow as tf

train_path = 'cats and dogs/train'
valid_path = 'cats and dogs/valid'
test_path = 'cats and dogs/test'

train_batches = tf.keras.preprocessing.image.ImageDataGenerator().flow_from_directory(train_path, target_size=(244, 244), classes=['dog', 'cat'], batch_size=10)
valid_batches = tf.keras.preprocessing.image.ImageDataGenerator().flow_from_directory(valid_path, target_size=(244, 244), classes=['dog', 'cat'], batch_size=5)
test_batches = tf.keras.preprocessing.image.ImageDataGenerator().flow_from_directory(test_path, target_size=(244, 244), classes=['dog', 'cat'], batch_size=5)
test_batches.class_indices # to see the indices : {'cat':0, 'dog': 1, 'lizard': 2} : [1. 0. 0.] -> cat


### Create and train the model

In [ ]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(244, 244, 3)), # number of output filter, the kernel size ( convo window), hight/width/channel(RGB)
	tf.keras.layers.Flatten(), # used to flat the output of the convo layer into a 1D tensor --> then fed into the dense layer
	tf.keras.layers.Dense(2, activation='softmax'),
	])
model.compile(tf.keras.optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_batches, steps_per_epoch=5, validation_data=validation_batches, validation_steps=4, epochs=5, verbose=2)
# fit_generator - to fit the model batch by batch ( because of the ImageDataGenerator)
# steps_per_epoch - total number of batches until a epoch is finished ( 50 / 10 = 5)
# validation steps - the same as for steps_per_epoch

### Importing an already trained model

In [ ]:
vgg16_model = tf.keras.applications.vgg16.VGG16()
# Because VGG is not a sequential model, we will take each layer and createa sequential model
model = tf.keras.models.Sequential()
for layer in vgg16_model.layers:
    model.add(layer)
model.layers.pop() # Delete that last 1000 outputs layer
for layer in model.layers:
    layer.trainable = False
model.add(tf.keras.layers.Dense(2, activation='softmax'))

### Data augmentation

In [ ]:
from scipy import misc, ndimage
import numpy as np
import matplotlib.pyplot as plt
import tensorflow

generator = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=10, # 10 radians
                                                            width_shift_range=0.1, # 0.1 fraction of the entire width of the image
                                                            height_shift_range=0.1,
                                                            shear_range=0.15,
                                                            zoom_range=0.1,
                                                            channel_shift_range=10.,
                                                            horizontal_flip=True)
image_path = 'man.png'
image = np.expand_dims(ndimage.imread(image_path), 0) # expand_dims to be compatible later on
aug_iter = gen.flow(image) # generate batches of augmented images: takes the numpy data and generates back augmented data
aug_images = [next(aug_iter)[0].astype(np.uint8) for i in range(10)]

### Initialize and access bias

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(4, input_shape=(1,), activation='relu', use_bias=True, bias_initializer='zeros'),
    tf.keras.layers.Dense(2, activation='softmax')
 ])
model.get_weights()

Trainable parameters:
 - weights and biases

## CNNs

- use of filters: Edge detector, squares, corners, circles
- Sliding through each 3x3 block of pixels: convolving through each block ( using a filter)
- Type of filters:
- -1: black, 1: white, 0: grey
- -1 -1 -1
-  1  1  1
-  0  0  0

In [ ]:
from tensorflow.keras.layers import Activation
from tensorflow.keras.convolutional import *
from tensorflow.keras.core import Dense, Flatten

### Zero padding
Initial<br>
1 1 3<br>
4 2 4<br>
2 4 9<br>  
Padding<br>
0 0 0 0 0<br>
0 1 1 3 0<br>
0 4 2 4 0<br>
0 2 4 9 0<br>
0 0 0 0 0

### CNN model example:

In [ ]:
model = tf.keras.models.Sequential([
    Dense(16, activation='relu', input_shape(20, 20, 3)),
    Conv2D(32, kernel_size(3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size(2, 2), strides=2, padding='valid'),
    Conv2D(64, kernel_size(5, 5), activation='relu', padding='same'), # padding
    Dense(2, activation='softmax')
])

- filter size = how much we pool
- stride = how much we move after we pool